# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), January 18, 2019**

In [1]:
using DataFrames

## Возможные подводные камни

### Знать, что копируется при создании `DataFrame`

In [2]:
x = DataFrame(rand(3, 5))

3×5 DataFrame
│ Row │ x1       │ x2       │ x3       │ x4         │ x5       │
│     │ Float64  │ Float64  │ Float64  │ Float64    │ Float64  │
├─────┼──────────┼──────────┼──────────┼────────────┼──────────┤
│ 1   │ 0.354313 │ 0.621338 │ 0.907431 │ 0.00580261 │ 0.856877 │
│ 2   │ 0.579985 │ 0.565304 │ 0.624031 │ 0.853468   │ 0.391311 │
│ 3   │ 0.620185 │ 0.231207 │ 0.340998 │ 0.386127   │ 0.452838 │

In [3]:
y = convert(DataFrame, x)
x === y # копирование не производится

true

In [4]:
y = copy(x)
x === y # не тот же объект

false

In [5]:
y = DataFrame(x)
x === y

false

In [6]:
all(x[i] === y[i] for i in ncol(x)) # но столбцы одинаковы

true

In [7]:
x = 1:3; y = [1, 2, 3]; df = DataFrame(x=x,y=y) 
# то же самое при создании массивов или назначении столбцов, кроме диапазонов

3×2 DataFrame
│ Row │ x     │ y     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 1     │ 1     │
│ 2   │ 2     │ 2     │
│ 3   │ 3     │ 3     │

In [8]:
y === df[:y] # тот же объект

true

In [9]:
typeof(x), typeof(df[:x]) # диапазон преобразуется в вектор

(UnitRange{Int64}, Array{Int64,1})

In [10]:
y === df[:, :y] # разрезание строк всегда создает копию

false

### Не изменяйте предка `GroupedDataFrame` или` view`

In [11]:
x = DataFrame(id=repeat([1,2], outer=3), x=1:6)
g = groupby(x, :id)

GroupedDataFrame with 2 groups based on key: id
First Group (3 rows): id = 1
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 1     │ 1     │
│ 2   │ 1     │ 3     │
│ 3   │ 1     │ 5     │
⋮
Last Group (3 rows): id = 2
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 2     │ 2     │
│ 2   │ 2     │ 4     │
│ 3   │ 2     │ 6     │

In [12]:
x[1:3, 1]=[2,2,2]
g # хорошо - это неправильно сейчас, g это просто view

GroupedDataFrame with 2 groups based on key: id
First Group (3 rows): id = 2
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 2     │ 1     │
│ 2   │ 2     │ 3     │
│ 3   │ 1     │ 5     │
⋮
Last Group (3 rows): id = 2
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 2     │ 2     │
│ 2   │ 2     │ 4     │
│ 3   │ 2     │ 6     │

In [13]:
s = view(x, 5:6, :)

2×2 SubDataFrame
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 1     │ 5     │
│ 2   │ 2     │ 6     │

In [14]:
deleterows!(x, 3:6)

2×2 DataFrame
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 2     │ 1     │
│ 2   │ 2     │ 2     │

In [15]:
s # error

BoundsError: BoundsError: attempt to access 2-element Array{Int64,1} at index [5:6]

### Помните, что вы можете фильтровать столбцы `DataFrame`, используя логические значения

In [16]:
using Random
Random.seed!(1)
x = DataFrame(rand(5, 5))

5×5 DataFrame
│ Row │ x1         │ x2       │ x3       │ x4        │ x5        │
│     │ Float64    │ Float64  │ Float64  │ Float64   │ Float64   │
├─────┼────────────┼──────────┼──────────┼───────────┼───────────┤
│ 1   │ 0.236033   │ 0.210968 │ 0.555751 │ 0.209472  │ 0.0769509 │
│ 2   │ 0.346517   │ 0.951916 │ 0.437108 │ 0.251379  │ 0.640396  │
│ 3   │ 0.312707   │ 0.999905 │ 0.424718 │ 0.0203749 │ 0.873544  │
│ 4   │ 0.00790928 │ 0.251662 │ 0.773223 │ 0.287702  │ 0.278582  │
│ 5   │ 0.488613   │ 0.986666 │ 0.28119  │ 0.859512  │ 0.751313  │

In [17]:
x[x[:x1] .< 0.25] 
# хорошо - мы отфильтровали столбцы, а не строки случайно, 
# так как вы можете выбрать столбцы, используя логические значения

5×2 DataFrame
│ Row │ x1         │ x4        │
│     │ Float64    │ Float64   │
├─────┼────────────┼───────────┤
│ 1   │ 0.236033   │ 0.209472  │
│ 2   │ 0.346517   │ 0.251379  │
│ 3   │ 0.312707   │ 0.0203749 │
│ 4   │ 0.00790928 │ 0.287702  │
│ 5   │ 0.488613   │ 0.859512  │

In [18]:
x[x[:x1] .< 0.25, :] # наверное это то что мы хотели

2×5 DataFrame
│ Row │ x1         │ x2       │ x3       │ x4       │ x5        │
│     │ Float64    │ Float64  │ Float64  │ Float64  │ Float64   │
├─────┼────────────┼──────────┼──────────┼──────────┼───────────┤
│ 1   │ 0.236033   │ 0.210968 │ 0.555751 │ 0.209472 │ 0.0769509 │
│ 2   │ 0.00790928 │ 0.251662 │ 0.773223 │ 0.287702 │ 0.278582  │

### Выбор столбца для DataFrame создает псевдонимы, если он явно не скопирован

In [19]:
x = DataFrame(a=1:3)
x[:b] = x[1] # alias
x[:c] = x[:, 1] # copy
x[:d] = x[1][:] # copy
x[:e] = copy(x[1]) # явная копия
display(x)
x[1,1] = 100
display(x)

3×5 DataFrame
│ Row │ a     │ b     │ c     │ d     │ e     │
│     │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │
├─────┼───────┼───────┼───────┼───────┼───────┤
│ 1   │ 1     │ 1     │ 1     │ 1     │ 1     │
│ 2   │ 2     │ 2     │ 2     │ 2     │ 2     │
│ 3   │ 3     │ 3     │ 3     │ 3     │ 3     │

3×5 DataFrame
│ Row │ a     │ b     │ c     │ d     │ e     │
│     │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │
├─────┼───────┼───────┼───────┼───────┼───────┤
│ 1   │ 100   │ 100   │ 1     │ 1     │ 1     │
│ 2   │ 2     │ 2     │ 2     │ 2     │ 2     │
│ 3   │ 3     │ 3     │ 3     │ 3     │ 3     │